In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
pd.read_csv('./Database/article_info.csv')

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN


In [3]:
pd.read_csv('./Database/view_log.csv')

,userID,articleID,userRegion,userCountry
0,USER_0000,ARTICLE_0661,NY,US
1,USER_0000,ARTICLE_2316,NY,US
2,USER_0000,ARTICLE_1345,NY,US
3,USER_0000,ARTICLE_1089,NY,US
4,USER_0000,ARTICLE_1484,NY,US
...,...,...,...,...
42712,USER_1420,ARTICLE_0682,SP,BR
42713,USER_1420,ARTICLE_2179,SP,BR
42714,USER_1420,ARTICLE_1848,SP,BR
42715,USER_1420,ARTICLE_0030,SP,BR


In [4]:
pd.read_csv('./Database/sample_submission.csv')

,userID,articleID
0,USER_0000,ARTICLE_0000
1,USER_0000,ARTICLE_0001
2,USER_0000,ARTICLE_0002
3,USER_0000,ARTICLE_0003
4,USER_0000,ARTICLE_0004
...,...,...
7070,USER_1420,ARTICLE_0000
7071,USER_1420,ARTICLE_0001
7072,USER_1420,ARTICLE_0002
7073,USER_1420,ARTICLE_0003


In [5]:
data = pd.read_csv('./Database/ncf_data.csv').fillna(0)
train, test = train_test_split(data, test_size=0.15, random_state=42)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)
df_0 = train.copy()

In [6]:
encoder_dict = {}

for col in ['p_id', 'u_id']:
    encoder_dict[col] = LabelEncoder()
    df_0[col] = encoder_dict[col].fit_transform(df_0[col])

In [7]:
df_0 = df_0.groupby(['u_id', 'p_id'], as_index=False).mean()
df_1 = df_0[df_0['u_id'].isin(df_0['u_id'].value_counts()[df_0['u_id'].value_counts() > 1].index)]

In [8]:
x = df_1.copy()
y = df_1['u_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y)

ratings_matrix = x_train.pivot(index='u_id', columns='p_id', values='u_rate')

In [9]:
matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix.index,
                               columns=ratings_matrix.index)

In [10]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))


def CF_simple(user_id, movie_id, ratings_matrix, user_similarity):
    if movie_id in ratings_matrix.columns:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = ratings_matrix[movie_id].copy()
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.dropna()
        sim_scores = sim_scores.drop(none_rating_idx)
        if sim_scores.sum() != 0:
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        else:
            mean_rating = 3.0  # 기본값
    else:
        mean_rating = 3.0

    return mean_rating

In [11]:
# 테스트 데이터의 user_id와 movie_id 간 pair를 맞춰 튜플형원소 리스트데이터를 만듬
id_pairs = zip(x_test['u_id'], x_test['p_id'])
# 모든 사용자-영화 짝에 대해서 주어진 예측모델에 의해 예측값 계산 및 리스트형 데이터 생성
y_pred = np.array([CF_simple(user, movie, ratings_matrix, user_similarity) for (user, movie) in id_pairs])
# 실제 평점값
y_true = np.array(x_test['u_rate'])

RMSE(y_true, y_pred)

np.float64(0.9599500503821221)